In [40]:
import os
os.environ["OMP_NUM_THREADS"] = "3"

import requests
from bs4 import BeautifulSoup
import zipfile 
import io 
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import time



def load_data():
    page_url='https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones'
    response=requests.get(page_url)
    if(response.status_code==200):
        soup=BeautifulSoup(response.content,'html.parser')
        link=soup.select_one('a[href$=".zip"]')['href']
        download_url='https://archive.ics.uci.edu'+link
        response_url=requests.get(download_url)
        if(response_url.status_code==200):
            with zipfile.ZipFile(io.BytesIO(response_url.content)) as outer_zip:
                inner_zip_name= 'UCI HAR Dataset.zip'
                with outer_zip.open(inner_zip_name) as inner_zip_file:
                    with zipfile.ZipFile(io.BytesIO(inner_zip_file.read())) as inner_zip:
                        with inner_zip.open('UCI HAR Dataset/train/X_train.txt') as myfile:
                            df=pd.read_csv(myfile,delim_whitespace=True,header=None)
                        with inner_zip.open('UCI HAR Dataset/train/y_train.txt') as myfile_y:
                            y=pd.read_csv(myfile_y,delim_whitespace=True, header=None)
        else:
            raise Exception("falied to download the file or parse the dataset.")
        return df,y

In [41]:
import os
os.environ["OMP_NUM_THREADS"] = "3"

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import time

#loading datasets 
df,y=load_data()

#Encode class labels
label_encoder=LabelEncoder()
encoded_y=label_encoder.fit_transform(y.values.ravel())

#Scale the features
scaler=StandardScaler()
df_scaled=scaler.fit_transform(df)

#Spit the data into training and testing sets for all the features
X_train_full, X_test_full, y_train, y_test=train_test_split(df_scaled,encoded_y,test_size=0.2, random_state=42)

import time
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

#NaiveBayes
start_time=time.time()
classifier_pipeline_full=Pipeline([
    ('classifier',GaussianNB())
                           ])
classifier_pipeline_full.fit(X_train_full,y_train)
y_full=classifier_pipeline_full.predict(X_test_full)
end_time=time.time()
full_time=end_time-start_time
accuracy=accuracy_score(y_test,y_full)


C:\Users\BIT\AppData\Local\Temp\ipykernel_13232\843175436.py:34: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df=pd.read_csv(myfile,delim_whitespace=True,header=None)
C:\Users\BIT\AppData\Local\Temp\ipykernel_13232\843175436.py:36: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y=pd.read_csv(myfile_y,delim_whitespace=True, header=None)


In [45]:
import os
os.environ["OMP_NUM_THREADS"] = "3"

from sklearn.cluster import KMeans
import numpy as np
# K-Means for Dimensionality-Reduction
n_clusters=50
kmeans=KMeans(n_clusters=n_clusters,random_state=42,n_init=10)
kmeans.fit(df_scaled.T)
selected_features_indices=[np.random.choice(np.where(kmeans.labels_ ==i)[0]) for i in range(n_clusters)]
selected_features=df_scaled[:,selected_features_indices]

#Split the data with reduces features
X_train_reduced, X_test_reduced = train_test_split(selected_features, test_size=0.2 , random_state=42)

C:\Users\BIT\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


In [46]:
import os
os.environ["OMP_NUM_THREADS"] = "3"

import time
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score


#Model woth reduced features using Naive Byes
start_time=time.time()
classifier_pipeline_reduced=Pipeline([
    ('classifier',GaussianNB())
                        ])
classifier_pipeline_reduced.fit(X_train_reduced, y_train)
y_reduce= classifier_pipeline_reduced.predict(X_test_reduced)
end_time=time.time()
reduce_time=end_time-start_time
accuracy_now= accuracy_score(y_test,y_reduce)

In [49]:
# Print comparison results
print("Baseline Model (All Features):")
print("Accuracy:", accuracy)
print("Training Time:", full_time, "seconds")
print("Number of Features:", X_train_full.shape[1])

print("\nModel with Reduced Features (K-Means):")
print("Accuracy:", accuracy_now)
print("Training Time:", reduce_time, "seconds")
print("Number of Features:", n_clusters)


#Final Result with Baseline Model and Model with Reduced Features

Baseline Model (All Features):
Accuracy: 0.7314751869476547
Training Time: 0.16648364067077637 seconds
Number of Features: 561

Model with Reduced Features (K-Means):
Accuracy: 0.8348062542488104
Training Time: 0.015496969223022461 seconds
Number of Features: 50
